<a href="https://colab.research.google.com/github/Jaewon45/BDAnalysis/blob/main/bostonhousing_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Regression Example 2 - Boston Housing

In [1]:
import tensorflow as tf; from tensorflow import keras
from tensorflow.keras import layers; from tensorflow.keras.layers import Dense

In [28]:
SEED = 22; random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)
from sklearn import datasets; boston_housing = datasets.load_boston()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

b) data preprocessing
- df.info()시 data가 수치형으로 인코딩돼있음 (float64) - 원핫인코딩 필요없음
- 범주형 변수 있지만 2class 변수는 크게 바꿀필요 없음 (하고있지만) 

In [3]:
df = pd.DataFrame(boston_housing.data, columns = boston_housing.feature_names)
df['MEDV'] = boston_housing.target # 주택가격. 타운의 주택가격중앙값(단위1000달러)

In [4]:
df['CHAS'].value_counts() # unbalanced data

0.0    471
1.0     35
Name: CHAS, dtype: int64

In [6]:
df = pd.get_dummies(data = df, columns = ['CHAS'], prefix = ['CHAS']) # 원핫

In [7]:
# df의 column 순서 변경
df = df[['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 
   'RAD', 'TAX','PTRATIO', 'B', 'LSTAT', 'CHAS_0.0', 'CHAS_1.0', 'MEDV']]
X = df.iloc[:, :-1]; Y = df.iloc[:, -1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    shuffle=True, random_state = SEED)

In [8]:
# feature scaling
from sklearn import preprocessing
minmax_scaler = preprocessing.MinMaxScaler();norm_fit = minmax_scaler.fit(X_train)
X_train_norm = norm_fit.transform(X_train);X_test_norm = norm_fit.transform(X_test)
standard_scaler = preprocessing.StandardScaler();stan_fit = standard_scaler.fit(X_train)
X_train_stan = stan_fit.transform(X_train);X_test_stan = stan_fit.transform(X_test)
# test data를 정규화하는 경우 train data의 통계량을 이용

    c) building model

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
def build_model(num_input=1):
    model = Sequential()
    # hidden layer
    model.add(Dense(128, activation='relu', input_dim=num_input))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    # output layer
    model.add(Dense(1, activation='linear'))
    # model compile
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return model
model = build_model(num_input=14)   # feature개수 = 14
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1920      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 1)                 17        
                                                                 
Total params: 12,801
Trainable params: 12,801
Non-trainable params: 0
_________________________________________________________________


In [10]:
### 3개의 은닉층 존재
### 구조도에서 레이어~레이어간 연결선은 sum of WiXi (i=1,..,p) 의미

    d) training model & Test model

- Min-max normalization

In [11]:
history = model.fit(X_train_norm, y_train, batch_size = 32,
                    epochs=200, validation_split = 0.2, verbose=0)  

In [27]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(3)

,loss,mse,val_loss,val_mse,epoch
197,7.689589,7.689589,10.914155,10.914155,197
198,7.401301,7.401301,10.555208,10.555208,198
199,6.603505,6.603505,9.412611,9.412611,199


In [26]:
res = model.evaluate(X_test_norm, y_test, verbose=2) # loss=res[0], mse=res[1]
print(f"테스트 세트의 MSE: {round(res[1],3)*1000}$") # 단위용 *1000$ 추가

41/41 - 0s - loss: 1.0899 - accuracy: 0.3646 - 287ms/epoch - 7ms/step
테스트 세트의 MSE: 365.0$


- 성능평가
 - 1. plot_loss_curve(total_epoch=100, start=1); plot_mse_curve(total_epoch=100, start=1)
 - => epoch 60정도부터 stabilized -> total_epoch=40으로 바꿔보기
 - 2. 모델의 성능을 확인을위한 fitted vs true value plot 확인 : 잘 적합된 case.  
 - + plot에서 이상치 확인 가능
 - + plt.xlim([0,plt.xlim()[1]]);plt.ylim([0,plt.ylim()[1]]);_=plt.plot([-100, 100], [-100, 100]) 추가
 - 3. R2값 확인 (Y의 변동량을 얼마나 설명하는지)
- 표준화 X도 동일과정 반복
 - 정규화보다 더 빠르게 loss가 수렴, MSE 감소 및 R2 증가